# Pandas - Cleaning data - casting datatypes and handling missing values

## Table of contents

* [Detecting missing values: `DataFrame.isnull()` or `DataFrame.isna()` methods](#Detecting-missing-values:-DataFrame.isnull()-or-DataFrame.isna()-methods)
* [Filling missing values: `DataFrame.fillna()` method](#Filling-missing-values:-DataFrame.fillna()-method)
* [Dropping missing values: `DataFrame.dropna()` method](#Dropping-missing-values:-DataFrame.dropna()-method)
* [Handling custom missing values](#Handling-custom-missing-values)
    * [Handling custom missing values at the `DataFrame` level: `DataFrame.replace()` method](#Handling-custom-missing-values-at-the-DataFrame-level:-DataFrame.replace()-method)
    * [Handling custom missing values at the `.csv` import level: `pandas.read_csv()` method's `na_values=` argument](#Handling-custom-missing-values-at-the-.csv-import-level:-pandas.read_csv()-method's-na_values=-argument)
* [Data types: `DataFrame.dtypes` attribute and Casting data types: `DataFrame.astype({dict})` or `Series.astype()` methods](#Data-types:-DataFrame.dtypes-attribute)
* [Now, let's go back to the stack overflow dataset and work with it](#Now,-let's-go-back-to-the-stack-overflow-dataset-and-work-with-it)
    

***

In [386]:
import pandas as pd
import numpy as np

In [387]:
people = {
    'first': ['Corey', 'Jane', 'John', 'Chris', np.nan, None, 'NA'], 
    'last': ['Schafer', 'Doe', 'Doe', 'Schafer', np.nan, np.nan, 'Missing'], 
    'email': ['CoreyMSchafer@gmail.com', 'JaneDoe@email.com', 'JohnDoe@email.com', None, np.nan, 'Anonymous@email.com', 'NA'],
    'age': ['33', '55', '63', '36', None, None, 'Missing']
}

Notice that we have added a few missing values to our dictionary above:
- `np.nan`: Not A Number (NAN) value from the `numpy` library
- `None` values
- Some custom missing values with the strings 'NA' and 'Missing' 

In [388]:
df_ppl = pd.DataFrame(people)

In [389]:
df_ppl

,first,last,email,age
0,Corey,Schafer,CoreyMSchafer@gmail.com,33
1,Jane,Doe,JaneDoe@email.com,55
2,John,Doe,JohnDoe@email.com,63
3,Chris,Schafer,None,36
4,NaN,NaN,NaN,None
5,None,NaN,Anonymous@email.com,None
6,NA,Missing,NA,Missing


## Detecting missing values: `DataFrame.isnull()` or `DataFrame.isna()` methods

Before we start handling the missing values, it is best to first 'detect' missing values in a `DataFrame`. That is what `DataFrame.isnull()` or `DataFrame.isna()` methods help us with.

We have looked at `DataFrame.isnull()` in the *'pandas_filtering'* notebook earlier.

`DataFrame.isna()` is identical to `DataFrame.isnull()`. These two methods do exactly the same thing. Even their docs are identical.

**But why have two methods with different names do the same thing?**

> This is because `pandas`' DataFrames are based on R's DataFrames. In R `na` and `null` are two separate things.

> However, in python, `pandas` is built on top of `numpy`, which has neither `na` nor `null` values. Instead `numpy` has `NaN` values (which stands for "Not a Number"). Consequently, `pandas` also uses `NaN` values.

Both, the `DataFrame.isnull()` or `DataFrame.isna()` methods, return a boolean same-sized object indicating if the values are missing.

In [390]:
df_ppl.isnull()

,first,last,email,age
0,False,False,False,False
1,False,False,False,False
2,False,False,False,False
3,False,False,True,False
4,True,True,True,True
5,True,True,False,True
6,False,False,False,False


In [391]:
df_ppl.isna()

,first,last,email,age
0,False,False,False,False
1,False,False,False,False
2,False,False,False,False
3,False,False,True,False
4,True,True,True,True
5,True,True,False,True
6,False,False,False,False


## Filling missing values: `DataFrame.fillna()` method

The `fillna()` method is used to fill `NaN` values with values of our choice (or values using the specified method).

Let's say that we have a `DataFrame` with student marks for some assignments. If a student does not submit an assignment, it is initially marked as `NaN`. However, while calculating grades, we want to replace these `NaN` values with 0 marks. That is where `fillna()`can be used: 

In [392]:
marks = {
    'name': ['Carl', 'Dave', 'Eva', 'Khalid', 'Ana', 'Ole', 'Kristin'], 
    'assignment_1': [8, 7, 5, 10, 6, 10, 8], 
    'assignment_2': [5, 7, np.nan, 8, 8, np.nan, np.nan],
    'assignment_3': [np.nan, 4, 3, 7, 4, 7, 6],
    'assignment_4': [np.nan, 3, 7, 7, np.nan, 8, 7],
    'assignment_5': [9, 6, np.nan, 9, 5, 9, np.nan],
}

In [393]:
df_marks = pd.DataFrame(marks)

In [394]:
df_marks

,name,assignment_1,assignment_2,assignment_3,assignment_4,assignment_5
0,Carl,8,5.0,NaN,NaN,9.0
1,Dave,7,7.0,4.0,3.0,6.0
2,Eva,5,NaN,3.0,7.0,NaN
3,Khalid,10,8.0,7.0,7.0,9.0
4,Ana,6,8.0,4.0,NaN,5.0
5,Ole,10,NaN,7.0,8.0,9.0
6,Kristin,8,NaN,6.0,7.0,NaN


In [395]:
df_marks.fillna(0, inplace=True)

In [396]:
df_marks

,name,assignment_1,assignment_2,assignment_3,assignment_4,assignment_5
0,Carl,8,5.0,0.0,0.0,9.0
1,Dave,7,7.0,4.0,3.0,6.0
2,Eva,5,0.0,3.0,7.0,0.0
3,Khalid,10,8.0,7.0,7.0,9.0
4,Ana,6,8.0,4.0,0.0,5.0
5,Ole,10,0.0,7.0,8.0,9.0
6,Kristin,8,0.0,6.0,7.0,0.0


This would then enable us to calculate a correct *'mean'* for each student.

In [397]:
df_marks['mean'] = df_marks.mean(axis='columns')

In [398]:
df_marks

,name,assignment_1,assignment_2,assignment_3,assignment_4,assignment_5,mean
0,Carl,8,5.0,0.0,0.0,9.0,4.4
1,Dave,7,7.0,4.0,3.0,6.0,5.4
2,Eva,5,0.0,3.0,7.0,0.0,3.0
3,Khalid,10,8.0,7.0,7.0,9.0,8.2
4,Ana,6,8.0,4.0,0.0,5.0,4.6
5,Ole,10,0.0,7.0,8.0,9.0,6.8
6,Kristin,8,0.0,6.0,7.0,0.0,4.2


## Dropping missing values: `DataFrame.dropna()` method

In [399]:
df_ppl

,first,last,email,age
0,Corey,Schafer,CoreyMSchafer@gmail.com,33
1,Jane,Doe,JaneDoe@email.com,55
2,John,Doe,JohnDoe@email.com,63
3,Chris,Schafer,None,36
4,NaN,NaN,NaN,None
5,None,NaN,Anonymous@email.com,None
6,NA,Missing,NA,Missing


In [400]:
df_ppl.dropna()

,first,last,email,age
0,Corey,Schafer,CoreyMSchafer@gmail.com,33
1,Jane,Doe,JaneDoe@email.com,55
2,John,Doe,JohnDoe@email.com,63
6,NA,Missing,NA,Missing


Note that `.dropna()` drops all the **ROWS** with **ANY** `NaN` or `None` values, but it does not drop the rows with *custom missing values* (i.e. the strings 'NA' and 'Missing').

Now, the `.dropna()` method above runs with some *implicit* default arguments. 

Let's re-run the `.dropna()` method with the same default arguments, but *explicit*:

In [401]:
df_ppl.dropna(axis='index', how='any')

,first,last,email,age
0,Corey,Schafer,CoreyMSchafer@gmail.com,33
1,Jane,Doe,JaneDoe@email.com,55
2,John,Doe,JohnDoe@email.com,63
6,NA,Missing,NA,Missing


The `axis='index', how='any'` arguments effectively mean: 'Drop **ROWS** with **ANY** missing values', respectively.

If we set these arguments to `axis='index', how='all'`, we would 'Drop **ROWS** with **ALL** missing values'.

In [402]:
df_ppl.dropna(axis='index', how='all')

,first,last,email,age
0,Corey,Schafer,CoreyMSchafer@gmail.com,33
1,Jane,Doe,JaneDoe@email.com,55
2,John,Doe,JohnDoe@email.com,63
3,Chris,Schafer,None,36
5,None,NaN,Anonymous@email.com,None
6,NA,Missing,NA,Missing


Now, let's say that the only value that is mandatory for us, and hence would be the criteria for including or dropping a row, is the *'email'*. Then we could use the `subset=` argument:

In [403]:
df_ppl

,first,last,email,age
0,Corey,Schafer,CoreyMSchafer@gmail.com,33
1,Jane,Doe,JaneDoe@email.com,55
2,John,Doe,JohnDoe@email.com,63
3,Chris,Schafer,None,36
4,NaN,NaN,NaN,None
5,None,NaN,Anonymous@email.com,None
6,NA,Missing,NA,Missing


In [404]:
df_ppl.dropna(axis='index', how='any', subset=['email'])

,first,last,email,age
0,Corey,Schafer,CoreyMSchafer@gmail.com,33
1,Jane,Doe,JaneDoe@email.com,55
2,John,Doe,JohnDoe@email.com,63
5,None,NaN,Anonymous@email.com,None
6,NA,Missing,NA,Missing


Now,since we have a `subset=` argument with only 1 column name passed into it, the `how=` argument becomes irrelevant. 

The `how=` argument would be relevant only when:
- Either there is NO `subset=` argument defined for the `.dropna()` method, i.e. the `.dropna()` applies to all the columns of the `DataFrame`, 
- Or the `subset=` argument is defined with MULTIPLE column names passed into it.

Now, if we want to drop all the **ROWS** with **ANY** of the *'last'* or *'email'* column values missing: 

In [405]:
df_ppl

,first,last,email,age
0,Corey,Schafer,CoreyMSchafer@gmail.com,33
1,Jane,Doe,JaneDoe@email.com,55
2,John,Doe,JohnDoe@email.com,63
3,Chris,Schafer,None,36
4,NaN,NaN,NaN,None
5,None,NaN,Anonymous@email.com,None
6,NA,Missing,NA,Missing


In [406]:
df_ppl.dropna(axis='index', how='any', subset=['last', 'email'])

,first,last,email,age
0,Corey,Schafer,CoreyMSchafer@gmail.com,33
1,Jane,Doe,JaneDoe@email.com,55
2,John,Doe,JohnDoe@email.com,63
6,NA,Missing,NA,Missing


On all the `.dropna()` executions above, we can see that it returns a `DataFrame` with the missing values dropped, but does not make changes *inplace*.

To make changes *inplace* we can just set the argument `inplace=True`. But we are not going to do that here.

## Handling custom missing values

We could handle the custom missing values at:
- Either the `DataFrame` level,
- Or the `.csv` import level.

### Handling custom missing values at the `DataFrame` level: `DataFrame.replace()` method

We can use the `DataFrame.replace()` method to replace custom missing values with `np.nan`.

In [407]:
people = {
    'first': ['Corey', 'Jane', 'John', 'Chris', np.nan, None, 'NA'], 
    'last': ['Schafer', 'Doe', 'Doe', 'Schafer', np.nan, np.nan, 'Missing'], 
    'email': ['CoreyMSchafer@gmail.com', 'JaneDoe@email.com', 'JohnDoe@email.com', None, np.nan, 'Anonymous@email.com', 'NA'],
    'age': ['33', '55', '63', '36', None, None, 'Missing']
}

In [408]:
df_a = pd.DataFrame(people)

In [409]:
df_a

,first,last,email,age
0,Corey,Schafer,CoreyMSchafer@gmail.com,33
1,Jane,Doe,JaneDoe@email.com,55
2,John,Doe,JohnDoe@email.com,63
3,Chris,Schafer,None,36
4,NaN,NaN,NaN,None
5,None,NaN,Anonymous@email.com,None
6,NA,Missing,NA,Missing


In [410]:
df_a.replace('NA', np.nan, inplace=True)
df_a.replace('Missing', np.nan, inplace=True)
df_a

,first,last,email,age
0,Corey,Schafer,CoreyMSchafer@gmail.com,33
1,Jane,Doe,JaneDoe@email.com,55
2,John,Doe,JohnDoe@email.com,63
3,Chris,Schafer,None,36
4,NaN,NaN,NaN,None
5,None,NaN,Anonymous@email.com,None
6,NaN,NaN,NaN,NaN


Now, since the *custom missing values* have been replaced with `np.nan`, we can re-run the `.dropna()` methods on the updated 
`DataFrame` above. We are not going to do that here.

### Handling custom missing values at the `.csv` import level: `pandas.read_csv()` method's `na_values=` argument

This way of handling custom missing values is demonstrated in the last section, where we import the *'survey_results_public.csv'* file.

## Data types: `DataFrame.dtypes` attribute
## and
## Casting data types: `DataFrame.astype({dict})` or `Series.astype()` methods

In [411]:
df_a

,first,last,email,age
0,Corey,Schafer,CoreyMSchafer@gmail.com,33
1,Jane,Doe,JaneDoe@email.com,55
2,John,Doe,JohnDoe@email.com,63
3,Chris,Schafer,None,36
4,NaN,NaN,NaN,None
5,None,NaN,Anonymous@email.com,None
6,NaN,NaN,NaN,NaN


`DataFrame.dtype` attribute returns a `Series` with the *data type* of each column. The result's `index` is the original `DataFrame`'s columns.

In [432]:
df_a.dtypes

first     object
last      object
email     object
age      float64
dtype: object

> **A brief explanation of the difference between Python `type()` and NumPy `dtype` objects:**<br><br>
Python defines only one type of a particular object (there is only one integer type, one floating-point type, etc.). This can be checked by using the `type()` function on the object.<br><br>
In `numpy`, there are 24 new fundamental Python types (each being a `dtype` object, that is an *instance* of the `numpy.dtype` class) to describe different types of scalars. A `dtype` object describes how the bytes in the fixed-size block of memory corresponding to an array item should be interpreted. These type descriptors are mostly based on the types available in the C-language, that CPython is written in, with several additional types compatible with Python’s types. The `dtype` object arrays in a `DataFrame` or a `Series` are checked with the `DataFrame.dtypes` or `Series.dtypes` attributes, respectively.<br><br>
For further reading on NumPy `dtype` objects: https://docs.scipy.org/doc/numpy-1.13.0/reference/arrays.dtypes.html#arrays-dtypes

Taking the *'mean'* of the *'age'* column from the `DataFrame` above, using `df_a['age'].mean()`, will give us a `TypeError` because the column data type is currently `str`, that does not have the `.mean()` method.

So we need to *'cast'* (convert) the *'age'* column to a *numeric* (`int` or `float`) data type. This is know as *'data type casting'*.

The question is: Should we convert the *'age'* column to `int` or `float` data type.

The caveat here is that, if the column that needs to be casted to a *numeric* data type, contains any `NaN` values, they have to be casted to the `float` data type. The reason for this is that, under the hood, the `np.nan` data type is `float`, and hence can not be casted to `int` (i.e. trying to cast `float` to `int `will result in a `TypeError`)

In [413]:
type(np.nan)

float

In [414]:
type(None)

NoneType

The `astype()` method is used to cast a pandas object to a specified `dtype`. The object returned has the same type as the caller.

We could use `DataFrame.astype({dict})` or `Series.astype()` methods.

In [415]:
df_a['age'] = df_a['age'].astype(float)

We could also called the `.astype()` method on the `DataFrame` object, as shown below (and commented out), with the same effect:

In [416]:
# df_a = df_a.astype({'age': float})

In [417]:
df_a.dtypes

first     object
last      object
email     object
age      float64
dtype: object

Now that the `dtype` of the *'age'* column has been casted to `float`, we can run the `.mean()` method on it:

In [418]:
df_a['age'].mean()

46.75

***

## Now, let's go back to the stack overflow dataset and work with it

In [419]:
df = pd.read_csv('work_directory/pandas/data/survey_results_public.csv', index_col='Respondent', na_values=['NA', 'Missing'])
df_schema = pd.read_csv('work_directory/pandas/data/survey_results_schema.csv', index_col='Column')

In [420]:
pd.set_option('display.max_rows', 85)
pd.set_option('display.max_columns', 85)

In [421]:
df.head()

,MainBranch,Hobbyist,OpenSourcer,OpenSource,Employment,Country,Student,EdLevel,UndergradMajor,EduOther,OrgSize,DevType,YearsCode,Age1stCode,YearsCodePro,CareerSat,JobSat,MgrIdiot,MgrMoney,MgrWant,JobSeek,LastHireDate,LastInt,FizzBuzz,JobFactors,ResumeUpdate,CurrencySymbol,CurrencyDesc,CompTotal,CompFreq,ConvertedComp,WorkWeekHrs,WorkPlan,WorkChallenge,WorkRemote,WorkLoc,ImpSyn,CodeRev,CodeRevHrs,UnitTests,PurchaseHow,PurchaseWhat,LanguageWorkedWith,LanguageDesireNextYear,DatabaseWorkedWith,DatabaseDesireNextYear,PlatformWorkedWith,PlatformDesireNextYear,WebFrameWorkedWith,WebFrameDesireNextYear,MiscTechWorkedWith,MiscTechDesireNextYear,DevEnviron,OpSys,Containers,BlockchainOrg,BlockchainIs,BetterLife,ITperson,OffOn,SocialMedia,Extraversion,ScreenName,SOVisit1st,SOVisitFreq,SOVisitTo,SOFindAnswer,SOTimeSaved,SOHowMuchTime,SOAccount,SOPartFreq,SOJobs,EntTeams,SOComm,WelcomeChange,SONewContent,Age,Gender,Trans,Sexuality,Ethnicity,Dependents,SurveyLength,SurveyEase
Respondent,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,I am a student who is learning to code,Yes,Never,The quality of OSS and closed source software ...,"Not employed, and not looking for work",United Kingdom,No,Primary/elementary school,NaN,"Taught yourself a new language, framework, or ...",NaN,NaN,4,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HTML/CSS;Java;JavaScript;Python,C;C++;C#;Go;HTML/CSS;Java;JavaScript;Python;SQL,SQLite,MySQL,MacOS;Windows,Android;Arduino;Windows,Django;Flask,Flask;jQuery,Node.js,Node.js,IntelliJ;Notepad++;PyCharm,Windows,I do not use containers,NaN,NaN,Yes,"Fortunately, someone else has that title",Yes,Twitter,Online,Username,2017,A few times per month or weekly,Find answers to specific questions;Learn how t...,3-5 times per week,Stack Overflow was much faster,31-60 minutes,No,NaN,"No, I didn't know that Stack Overflow had a jo...","No, and I don't know what those are",Neutral,Just as welcome now as I felt last year,Tech articles written by other developers;Indu...,14.0,Man,No,Straight / Heterosexual,NaN,No,Appropriate in length,Neither easy nor difficult
2,I am a student who is learning to code,No,Less than once per year,The quality of OSS and closed source software ...,"Not employed, but looking for work",Bosnia and Herzegovina,"Yes, full-time","Secondary school (e.g. American high school, G...",NaN,Taken an online course in programming or softw...,NaN,"Developer, desktop or enterprise applications;...",NaN,17,NaN,NaN,NaN,NaN,NaN,NaN,I am actively looking for a job,I've never had a job,NaN,NaN,Financial performance or funding status of the...,"Something else changed (education, award, medi...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C++;HTML/CSS;Python,C++;HTML/CSS;JavaScript;SQL,NaN,MySQL,Windows,Windows,Django,Django,NaN,NaN,Atom;PyCharm,Windows,I do not use containers,NaN,Useful across many domains and could change ma...,Yes,Yes,Yes,Instagram,Online,Username,2017,Daily or almost daily,Find answers to specific questions;Learn how t...,3-5 times per week,Stack Overflow was much faster,11-30 minutes,Yes,A few times per month or weekly,"No, I knew that Stack Overflow had a job board...","No, and I don't know what those are","Yes, somewhat",Just as welcome now as I felt last year,Tech articles written by other developers;Indu...,19.0,Man,No,Straight / Heterosexual,NaN,No,Appropriate in length,Neither easy nor difficult
3,"I am not primarily a developer, but I write co...",Yes,Never,The quality of OSS and closed source software ...,Employed full-time,Thailand,No,"Bachelor’s degree (BA, BS, B.Eng., etc.)",Web development or web design,"Taught yourself a new language, framework, or ...",100 to 499 employees,"Designer;Developer, back-end;Developer, front-...",3,22,1,Slightly satisfied,Slightly satisfied,Not at all confident,Not sure,Not sure,"I’m not actively looking, but I am open to new...",1-2 years ago,Int

Now, let's say we want to find out the **average years of coding experience for the respondents** in our `DataFrame`.

In [422]:
df['YearsCode'].head(10)

Respondent
1       4
2     NaN
3       3
4       3
5      16
6      13
7       6
8       8
9      12
10     12
Name: YearsCode, dtype: object

In [423]:
df['YearsCode'].dtypes

dtype('O')

In [424]:
type('O')

str

`dtype('O')` in `numpy` is a `str` object in Python. (Read: 'A brief explanation of the difference between Python `type()` and NumPy `dtype` objects')

Calling the `.mean()` method on the *'YearsCode'* column would return a `TypeError` because the `dtype`for this column is a *string* object.

Therefore, we need to *cast* the data type of this column to `float`.

In [425]:
# df['YearsCode'].astype(float)

The command commented in above will raise a `TypeError` because there is a string: 'Less than 1 year', present in the values of the column.

We could not *cast* the data type of the *'YearsCode'* column to `float` because there is one (or more) string(s) present in the values of the column.

Let's have a look at all the unique values in the column before attempting to *cast* it to `float` again.

We could use the `.value_counts()` method for a tally of the unique values, or we could simply use the `.unique()` method if we just want to see all the unique values without their count.

In [426]:
df['YearsCode'].unique()

array(['4', nan, '3', '16', '13', '6', '8', '12', '2', '5', '17', '10',
       '14', '35', '7', 'Less than 1 year', '30', '9', '26', '40', '19',
       '15', '20', '28', '25', '1', '22', '11', '33', '50', '41', '18',
       '34', '24', '23', '42', '27', '21', '36', '32', '39', '38', '31',
       '37', 'More than 50 years', '29', '44', '45', '48', '46', '43',
       '47', '49'], dtype=object)

Let's just replace the unique `str` values with the most representative *numeric* values.

Note: we do not have to do anything about the `NaN` values in this column since the `.mean()` method would ignore these when calculating the *mean*.

In [427]:
df['YearsCode'].replace('Less than 1 year', 0, inplace=True)

In [428]:
df['YearsCode'].replace('More than 50 years', 51, inplace=True)

In [429]:
df['YearsCode'].unique()

array(['4', nan, '3', '16', '13', '6', '8', '12', '2', '5', '17', '10',
       '14', '35', '7', 0, '30', '9', '26', '40', '19', '15', '20', '28',
       '25', '1', '22', '11', '33', '50', '41', '18', '34', '24', '23',
       '42', '27', '21', '36', '32', '39', '38', '31', '37', 51, '29',
       '44', '45', '48', '46', '43', '47', '49'], dtype=object)

In [433]:
df['YearsCode'] = df['YearsCode'].astype(float)

In [434]:
df['YearsCode'].dtypes

dtype('float64')

In [431]:
df['YearsCode'].mean()

11.647779751332148

So the **average years of coding experience for the respondents is about 11.6 years**.